# Transferring files over SSH (Secure Shell) using Python

By Kyle Baacke
3/13/2022

## Description:
  You may want to transfer a file or list of files to or from the cluster without using a physical drive. This can be accomplished over SSH using SCP. Alternatively, you can run SCP via python in order to streamline your analysis or data management processes. This snippet provides examples of how this can be accomplished.

## Helpful Links:
  https://stackoverflow.com/questions/250283/how-to-scp-in-python
  https://pypi.org/project/scp/
  https://pypi.org/project/scpclient/

## 0) Setup

In [1]:
import os
import shutil
import datetime as dt
import paramiko
from scp import SCPClient
import zipfile # Only required if you want to zip before sending
import getpass # Used for getting password from the user in a prompt rather than having your password in the code (which is a very, very bad practice)



## 1) Create function to instantiate a SSH Client to connect to the remote machine

In [2]:
def createSSHClient(server, port, user, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(server, port, user, password)
    return client

## 2) Choose a file pattern or list of patterns to transfer

  This sets up some strings to keep track of the dicetories in both the local and remote systems. *accuracy_path*  points to a specific file.

In [3]:
# The following line is ot enable the script to work on all opreating systems by dynamically assigning what the file deliminer will be.
sep = os.path.sep

# This line is to specify what the deliminer is on the remote system. Since the cluster is linux, the following should always work
remote_sep = '/'

# Identify the base of the paths for the local directory and the remote directory
base = f'S:\\'
remote_base = '/mnt/usb1'

output_path = '{base}{sep}MSC-HCP_analysis{sep}'

## 3) Connect to SSH Session

In [4]:
# Store username as a variable
username = 'kbaacke'
# Save hostname as a variable
hostname = 'r2.psych.uiuc.edu'
# Start SSH Session
# Request for password, without displaying the passowrd or soring it as a variable outside of the function call
ssh = createSSHClient(f'{hostname}', 22, f'{username}', getpass.getpass(f'Password for {username}@{hostname}:'))
# Start SCP Client on ssh client
scp = SCPClient(ssh.get_transport())

## 4) Transfer files

### 4.1) Transfer single files
  Use scp.get(*target_on_remote*, *target_on_local*) to transfer a single file from the remote device to your local device.
  Use scp.get(*target_on_local*, *target_on_remote*) to transfer a single file from your local device to the remote device.
  It is convenient to run this in a loop in case you want to add other files later witout 

In [7]:
output_path = '{base}{sep}MSC-HCP_analysis{sep}'
accuracy_path = output_path + 'Prediction_Accuracies.csv'
local_path = accuracy_path.format(base = base, sep=sep)
remote_path = accuracy_path.format(base = remote_base, sep=remote_sep)
scp.get(remote_path, local_path)
print(f'{hostname}:{remote_path} saved locally as {local_path}')

r2.psych.uiuc.edu:/mnt/usb1/MSC-HCP_analysis/Prediction_Accuracies.csv saved locally as S:\\MSC-HCP_analysis\Prediction_Accuracies.csv


  It is convenient to run this in a loop in case you want to add other files later witout too much effort.

In [8]:
files_to_transfer = [
  accuracy_path
]
for filename in files_to_transfer:
  try:
    local_path = filename.format(base = base, sep=sep)
    remote_path = filename.format(base = remote_base, sep=remote_sep)
    scp.get(remote_path, local_path)
    print(f'{hostname}:{remote_path} saved locally as {local_path}')
  except Exception as e:
    print(f'Error transferring {hostname}:{remote_path} to local:{local_path}: {e}')
  

r2.psych.uiuc.edu:/mnt/usb1/MSC-HCP_analysis/Prediction_Accuracies.csv saved locally as S:\\MSC-HCP_analysis\Prediction_Accuracies.csv


### 4.2) Transfer Directories
  You can also choose to transfer entire directories. To do this, you have two options: you can transfer the folder using the recursive option (*4.2.1*) or by compressing the files before transfer(*4.2.2*). Either way, you need patterns for the folders that you want to transfer.

In [ ]:
output_path = '{base}{sep}MSC-HCP_analysis{sep}'
metapath = output_path + 'metadata'
confpath =  output_path + 'confusion'
classpath = output_path + 'classification'
inputpath = output_path + 'inputs'

folder_patterns = [metapath, confpath, classpath, inputpath]


#### 4.2.1) Transfer Recursively

In [ ]:
for foldername in folder_patterns:
  try:
    local_path = foldername.format(base = base, sep=sep)
    remote_path = foldername.format(base = remote_base, sep=remote_sep)
    scp.get(remote_path, local_path, recursive=True)
    print(f'{hostname}:{remote_path}{remote_sep}* saved locally as {local_path}{sep}*')
  except Exception as e:
    print(f'Error transferring {hostname}:{remote_path}{remote_sep}* to local:{local_path}{sep}*: {e}')

### 4.2.2) Compress before sending

In [ ]:
base = f'S:\\Code\\'
remote_base = '/mnt/usb1/Code/'
archive_pattern = '{base}MSC_HCP.zip'
# Compress the file and save with the .zip extension
shutil.make_archive(f'{base}MSC_HCP', 'zip', f'{base}MSC_HCP')
# Transfer Archive
local_path = archive_pattern.format(base = base, sep=sep) + '.zip'
remote_path = archive_pattern.format(base = remote_base, sep=remote_sep) + '.zip'
scp.put(local_path, remote_path)